In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
# from gpt_index.composability.joint_qa_summary import QASummaryGraphBuilder
from llama_index import SimpleDirectoryReader, ServiceContext, LLMPredictor
from llama_index import GPTSimpleVectorIndex, GPTListIndex, GPTSimpleKeywordTableIndex
from llama_index.composability import ComposableGraph
from langchain.chat_models import ChatOpenAI


#### Load Documents

In [ ]:
reader = SimpleDirectoryReader('/Users/vince/Downloads/cw1', recursive=True)
documents = reader.load_data()


#### Parse into Nodes

In [ ]:
from llama_index.node_parser import SimpleNodeParser
nodes = SimpleNodeParser().get_nodes_from_documents(documents)

#### Add to Docstore

In [ ]:
from llama_index.docstore import DocumentStore
docstore = DocumentStore()
docstore.add_documents(nodes)

#### Define Multiple Indexes

Each index uses the same underlying Node.

In [ ]:
list_index = GPTListIndex(nodes, docstore=docstore)

In [ ]:
vector_index = GPTSimpleVectorIndex(nodes, docstore=docstore) 

In [ ]:
keyword_table_index = GPTSimpleKeywordTableIndex(nodes, docstore=docstore) 

In [ ]:
# NOTE: the docstore sitll has the same nodes
len(docstore.docs)

#### Test out some Queries

In [23]:
llm_predictor_chatgpt = LLMPredictor(llm=ChatOpenAI(temperature=0, max_tokens=4000))
service_context_chatgpt = ServiceContext.from_defaults(llm_predictor=llm_predictor_chatgpt)

Unknown max input size for gpt-3.5-turbo, using defaults.


In [24]:
from IPython.display import display
from IPython.display import Markdown


display(Markdown(str(vector_index.query(
"""                                     
Respond to the following request via code using the TypeScript language. Please wrap your response in triple backticks. Before responding, think through your solution step-by-step and ensure that it satisfies the request completely and comprehensively. Request:

Similar to what the `mapIsf` function does, create a function to map transport jobs.
"""
))))

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 3153 tokens
> [query] Total LLM token usage: 3153 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 111 tokens
> [query] Total embedding token usage: 111 tokens



```
const mapTransportJob = (transportJob: Cw1TransportJob, orgIds: OrgIdsMap): LxbTransportJobGraph => {
    const addInfoMap = mapAdditionalInfo(transportJob.AddInfoCollection)
    const format = addInfoMap['EntryType'] ? parseInt(addInfoMap['EntryType']) : null

    const orgAddressCollection = transportJob.OrganizationAddressCollection || null

    const shipmentType = addInfoMap['TransportJobShipmentType'] ?
        parseInt(addInfoMap['TransportJobShipmentType']) :
        null

    return {
        transportJob: {
            referenceId: transportJob.DataContext.DataSourceCollection.DataSource.Key,
            uinkCarrierSCAC: addInfoMap['UI_NKCarrierSCAC'],
            buyerAddress: mapCw1Address(
                findOrgAddress(orgAddressCollection, Cw1AddressType.BUYING_PARTY),
            ),
            consigneeEntity: mapIsfEntity(addInfoMap, 'Consignee'),
            importerEntity: map